In [ ]:
from dotenv import load_dotenv
import os  # provides ways to access the Operating System and allows us to read the environment variables

load_dotenv()

GOOGLE_AI_KEY = os.getenv("GOOGLE_AI_KEY_021124")
GOOGLE_API_KEY = GOOGLE_AI_KEY

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are world class technical documentation writer."),
        ("user", "{input}"),
    ]
)

llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=GOOGLE_API_KEY,
    convert_system_message_to_human=True,
)

chain = prompt | llm

In [7]:
from rich import print
resp = chain.invoke({"input": "how can langsmith help with testing?"})
print(resp.content)

1. **Automated Testing:** 
Langsmith can be integrated with automated testing frameworks to generate comprehensive test cases and test 
scripts. This helps in identifying potential issues early in the development cycle, reducing the need for manual 
testing.


2. **User Interface Testing:** 
Langsmith can generate user interface (UI) test cases based on the documentation. This helps in verifying the 
functionality, usability, and accessibility of the UI, ensuring that it meets the design specifications.


3. **API Testing:** 
Langsmith can generate API test cases based on the API documentation. This helps in testing the functionality, 
reliability, and security of the APIs, ensuring that they are working as expected.


4. **Performance Testing:** 
Langsmith can generate performance test cases to evaluate the system's performance under different load conditions.
This helps in identifying bottlenecks and optimizing the system for better performance.


5. **Security Testing:** 
Langsmith can generate security test cases based on the security requirements and documentation. This helps in 
identifying vulnerabilities and weaknesses in the system, ensuring that it is secure against potential attacks.


6. **Regression Testing:** 
Langsmith can generate regression test cases to verify that new changes or updates to the system do not break 
existing functionalities. This helps in maintaining the stability and reliability of the system.


7. **Exploratory Testing:** 
Langsmith can generate exploratory testing scenarios to help testers explore the system in a structured manner. 
This helps in discovering hidden defects and identifying areas for improvement.


8. **Test Case Prioritization:** 
Langsmith can help prioritize test cases based on their impact, risk, and coverage. This helps in optimizing the 
testing effort and focusing on the most critical areas first.


9. **Test Case Management:** 
Langsmith can be used to manage test cases, track their status, and monitor the overall testing progress. This 
helps in ensuring that all test cases are executed and that the testing process is efficient and effective.


10. **Test Reporting:** 
Langsmith can generate comprehensive test reports that summarize the testing results, identify defects, and provide
insights into the overall quality of the system. This helps stakeholders understand the testing outcomes and make 
informed decisions.

In [8]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [9]:
chain = prompt | llm | output_parser

In [10]:
resp = chain.invoke({"input": "how can langsmith help with testing?"})
print(resp)

1. **Automated Testing:** Langsmith can generate automated tests from natural language requirements. This can save 
time and effort for testers, as they don't have to manually create tests.

2. **Test Documentation:** Langsmith can generate test documentation from natural language requirements. This can 
help testers understand the requirements and how to test them.

3. **Test Case Generation:** Langsmith can generate test cases from natural language requirements. This can help 
testers create test cases that cover the requirements.

4. **Test Report Generation:** Langsmith can generate test reports from the results of automated tests. This can 
help testers understand the results of the tests and identify any areas that need improvement.

5. **Integration with Testing Tools:** Langsmith can integrate with testing tools such as Jira, TestRail, and 
Selenium. This can help testers manage and track their testing activities.

6. **Natural Language Processing:** Langsmith uses natural language processing (NLP) to understand the requirements
and generate tests. This makes it easy for testers to use Langsmith, even if they don't have a technical 
background.

7. **Collaboration:** Langsmith allows multiple testers to collaborate on test creation and execution. This can 
help testers share ideas and improve the quality of the tests.

8. **Continuous Testing:** Langsmith can be used for continuous testing. This means that tests can be run 
automatically as the code changes, which can help to identify defects early in the development process.

9. **Scalability:** Langsmith can be used to test large and complex systems. This is because Langsmith can generate
tests automatically, which can save time and effort for testers.

10. **Cost-Effectiveness:** Langsmith is a cost-effective way to improve the quality of testing. This is because 
Langsmith can automate the testing process and generate tests from natural language requirements.

In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

# Using HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

In [13]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(
    documents,
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"),
)

In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

document_chain = create_stuff_documents_chain(llm, prompt)

In [15]:
from langchain_core.documents import Document

document_chain.invoke(
    {
        "input": "how can langsmith help with testing?",
        "context": [
            Document(page_content="langsmith can let you visualize test results")
        ],
    }
)

'Langsmith can help with testing by letting you visualize test results.'

In [17]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [18]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by allowing users to run a chain over data points and visualize the outputs, assign
feedback to runs and mark them as correct or incorrect directly in the web app, and use a set of evaluators to 
evaluate the results once the test run completes. Additionally, LangSmith makes it easy to manually review and 
annotate runs through annotation queues, which can be used to assess subjective qualities that automatic evaluators
struggle with, like creativity or humor, and to sample and validate a subset of runs that were auto-evaluated.

##**Run Converstation Retrieval chain with history**
The provided way in LangChain quickstart guide throws error when we use Google Gemini with LangChain. There might be support issue of Langchain with Google Gemini. This is alternative method.

In [19]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=100)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2'))

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [20]:
from langchain.chains import ConversationalRetrievalChain

# Connect query to FAISS index using a retriever
retriever = vector.as_retriever()

qa_chain = ConversationalRetrievalChain.from_llm(
    llm, retriever, return_source_documents=True
)
print(retriever)

VectorStoreRetriever(
    tags=['FAISS', 'HuggingFaceEmbeddings'],
    vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7d3df670d6d0>
)

In [21]:
import sys

chat_history = []
while True:
    query = input("Prompt: ")
    result = qa_chain({"question": query, "chat_history": chat_history})
    print("Answer: " + result["answer"] + "\n")
    chat_history.append((query, result["answer"]))

/home/daryl/.asdf/installs/python/3.11.3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Answer: I do not have enough information to answer this question. The provided context does not contain any 
information about integrating LangSmith with a web application.

Answer: I cannot find the answer to your question in the provided context.

Answer: I cannot find the answer to your question in this document.

Answer: I cannot find the answer to your question in the context you provided.

...

Answer: I cannot answer this question as the provided context does not contain any information about how LangSmith 
can be integrated with a web application.

Answer: I cannot find the answer to your question in the context you provided.

Answer: I cannot find the answer to your question in the context you provided.

Answer: I cannot find the answer to your question in the provided context.

Answer: I'm sorry, but the provided context does not contain information on how to integrate LangSmith with a web 
application. Therefore, I cannot answer this question.